In [58]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q -U bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
from datasets import load_dataset
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
import torchvision.transforms as transforms

In [3]:
from google.colab import output
output.enable_custom_widget_manager()
!pip install huggingface_hub

import os
os.environ['HF_TOKEN'] = '' # change token


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# checkpoint = "HuggingFaceM4/tiny-random-idefics"
checkpoint = "HuggingFaceM4/idefics-9b"

# Here we skip some special modules that can't be quantized properly
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"],
)

processor = AutoProcessor.from_pretrained(checkpoint, use_auth_token=True)
# Simply take-off the quantization_config arg if you want to load the original model
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/processing_auto.py:224: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00018-of-00019.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Instantiating IdeficsAttention without passing a `layer_idx` is not recommended and will lead to errors during the forward call if caching is used. Please make sure to provide a `layer_idx` when creating this class.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
def check_inference(model, processor, prompts, max_new_tokens=3):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=True)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
import os
import pandas as pd
import random
from sklearn.metrics import accuracy_score, f1_score


base_path = '/content/drive/My Drive/Dataset/test'
all_folders = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]


In [9]:
all_folders

['CCAH',
 'ACCFP',
 'CCUIM',
 'CCSAD',
 'SCCC',
 'EIB',
 'GGCC',
 'TICC',
 'WICC',
 'EWCC']

In [10]:
import os
import pandas as pd

all_image_caption_pairs = []

for folder in all_folders:
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        frames_folder = None
        csv_file = None
        for item in os.listdir(folder_path):
            if item.endswith("_frames"):
                frames_folder = item
            elif item == f"{folder}.csv":
                csv_file = item

        if frames_folder and csv_file:
            frames_path = os.path.join(folder_path, frames_folder)
            csv_file_path = os.path.join(folder_path, csv_file)

            annotations = pd.read_csv(csv_file_path)

            folder_image_caption_pairs = []
            counter = 1
            for index, row in annotations.iterrows():
                image_file = os.path.join(frames_path, f'{folder}-{counter:03d}.jpg')
                if os.path.exists(image_file) and image_file.endswith('.jpg'):
                    caption = row.iloc[-1]
                    # print(caption)
                    if pd.isna(caption) or caption.endswith(']') or caption.strip() == '':
                        caption = 'NONE'
                    else:
                        caption = caption.lower()

                    label = row['label'] if 'label' in row else 'None'
                    # print(label)

                    folder_image_caption_pairs.append({"image": image_file, "caption": caption, "label": label})
                    counter += 1

            all_image_caption_pairs.append(folder_image_caption_pairs)

In [17]:
all_image_caption_pairs

[[{'image': '/content/drive/My Drive/Dataset/test/CCAH/CCAH_frames/CCAH-001.jpg',
   'caption': ' the crucial connection: climate change and health | kaiser permanente',
   'label': 1},
  {'image': '/content/drive/My Drive/Dataset/test/CCAH/CCAH_frames/CCAH-002.jpg',
   'caption': ' we know climate-related changes on our planet cause extreme weather.',
   'label': 1},
  {'image': '/content/drive/My Drive/Dataset/test/CCAH/CCAH_frames/CCAH-003.jpg',
   'caption': ' and extreme weather has life-changing effects.',
   'label': 2},
  {'image': '/content/drive/My Drive/Dataset/test/CCAH/CCAH_frames/CCAH-004.jpg',
   'caption': ' we see this in the rise of weather-related disasters around the globe.',
   'label': 2},
  {'image': '/content/drive/My Drive/Dataset/test/CCAH/CCAH_frames/CCAH-005.jpg',
   'caption': ' but there’s one aspect of this topic we don’t hear much about: how does climate change affect our health?',
   'label': 0},
  {'image': '/content/drive/My Drive/Dataset/test/CCAH/CC

In [18]:
all_image_caption_pairs[2][0]

{'image': '/content/drive/My Drive/Dataset/test/CCUIM/CCUIM_frames/CCUIM-001.jpg',
 'caption': ' climate change unlikely to increase malaria burden in west africa',
 'label': 1}

In [19]:
import pandas as pd
from PIL import Image
from io import BytesIO
import re

results = []
folder_pairs = all_image_caption_pairs[2]
for item in folder_pairs:
    frame_path = item['image']
    frame = Image.open(frame_path)
    subtitle = item['caption']
    label = item['label']

    prompts = [
        [
            frame,
            f"{subtitle}. Question: What is the stance of this image-text pair towards climate change? Choose one between 0 for neutral, 1 for support, and 2 for oppose.\n"
            "Answer: "
        ],
    ]

    generated_text = check_inference(model, processor, prompts)
    print(generated_text)

    match = re.search(r"Answer: (\d)", generated_text)
    if match:
        answer = match.group(1)
    else:
        answer = "Unknown"

    # print(answer)

    results.append([label, subtitle, answer])


results_df = pd.DataFrame(results, columns=['Label', 'Subtitle', 'Answer'])

results_df.to_csv('CCUIM_IDEFICS.csv', index=False)



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


climate change unlikely to increase malaria burden in west africa. Question: What is the stance of this image-text pair towards climate change? Choose one between 0 for neutral, 1 for support, and 2 for oppose.
Answer: 0

NONE. Question: What is the stance of this image-text pair towards climate change? Choose one between 0 for neutral, 1 for support, and 2 for oppose.
Answer: 1

many of the predicted impacts of climate change on the planet are bad if not catastrophic.. Question: What is the stance of this image-text pair towards climate change? Choose one between 0 for neutral, 1 for support, and 2 for oppose.
Answer: 0

melting glaciers. rising sea levels. more extreme weather.. Question: What is the stance of this image-text pair towards climate change? Choose one between 0 for neutral, 1 for support, and 2 for oppose.
Answer: 1

but what about the effect on diseases spread by mosquitoes?. Question: What is the stance of this image-text pair towards climate change? Choose one betwee

In [21]:
df = pd.read_csv('CCUIM_IDEFICS.csv')
df

,Label,Subtitle,Answer
0,1,climate change unlikely to increase malaria b...,0
1,0,NONE,1
2,2,many of the predicted impacts of climate chan...,0
3,2,melting glaciers. rising sea levels. more ext...,1
4,0,but what about the effect on diseases spread ...,1
5,0,will malaria become more common too? scientif...,1
6,0,now using a sophisticated model incorporating...,1
7,0,now using a sophisticated model incorporating...,1
8,0,their models predict that the number of malar...,0
9,2,because malaria is a disease of temperature-s...,1


In [22]:
accuracy = accuracy_score(df['Label'], df['Answer'])
print(f"Accuracy: {accuracy}")

f1 = f1_score(df['Label'], df['Answer'], average='macro')
print(f"F1 Score: {f1}")

Accuracy: 0.16666666666666666
F1 Score: 0.1384436701509872
